In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn import metrics
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset
import numpy as np
import copy
import statistics
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

import sys
sys.path.append('./../src')
import globals
from model import Net
from training import train_model, train_model_CL
from visualizations import plot_embeddings, plot_confusion_matrix
from feature_attribution import Feature_Importance_Evaluations
from pytorch_utils import get_features, get_labels
from embedding_measurements import measure_embedding_confusion_knn, measure_embedding_drift

In [ ]:
import submitit
import os
partition_nr = 1
partition = 'mlhiwidlc_gpu-rtx2080' if partition_nr == 0 else "mlhiwidlc_gpu-rtx2080-advanced" if partition_nr == 1 else "testdlc_gpu-rtx2080"

timelimit_hours = 2
num_gpus_per_node = 1
num_jobs = 1

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
log_path = os.path.join(parent_dir, 'submitit_logs')
run_path = os.path.join(parent_dir, 'src')


ex_parallel = submitit.AutoExecutor(folder=log_path)
ex_parallel.update_parameters(
        slurm_signal_delay_s=180,           # time to pass the USR2 signal to slurm before the job times out so that it can finish the run
        tasks_per_node=num_gpus_per_node,
        nodes=1,
        slurm_partition=partition,
        timeout_min=int(timelimit_hours*60),
        cpus_per_task=8,
        slurm_gres=f'gpu:{num_gpus_per_node}',
        slurm_setup = [f'export PYTHONPATH="${{PYTHONPATH}}:{run_path}"'],
    )

SEED = globals.SEED
DEVICE = globals.DEVICE  # torch.device("cuda:0")
full_trainset = globals.full_trainset
trainset = globals.trainset
testset = globals.testset
trainloaders = globals.trainloaders
valloaders = globals.valloaders
testloaders = globals.testloaders

In [4]:
# This is the two-step process used to prepare the
# data for use with the convolutional neural network.

# First step is to convert Python Image Library (PIL) format
# to PyTorch tensors.

# Second step is used to normalize the data by specifying a 
# mean and standard deviation for each of the three channels.
# This will convert the data from [0,1] to [-1,1]

# Normalization of data should help speed up conversion and
# reduce the chance of vanishing gradients with certain 
# activation functions.
def initialize_data():
    transform = transforms.Compose([
        transforms.ToTensor()
        #transforms.Normalize((0.5,), (0.5,))  # Normalizes to mean 0.5 and std 0.5 for the single channel
    ])

    globals.full_trainset = torchvision.datasets.MNIST('./../data/', train=True, download=True,
                                transform=transform)
    targets = np.array(globals.full_trainset.targets)

    if globals.val_set_size != 0:
        # Perform stratified split
        train_indices, val_indices = train_test_split(
            np.arange(len(targets)),
            test_size=0.01,
            stratify=targets
        )
    else:
        train_indices = np.arange(len(targets))
        val_indices = []

    # Create subsets
    valset = Subset(globals.full_trainset, val_indices)
    globals.trainset = Subset(globals.full_trainset, train_indices)

    globals.testset = torchvision.datasets.MNIST('./../data/', train=False, download=True,
                                transform=transform)

    # Define class pairs for each subset
    class_pairs = [tuple(range(i*globals.CLASSES_PER_ITER,(i+1)*globals.CLASSES_PER_ITER)) for i in range(globals.ITERATIONS)]
    #print(class_pairs)

    # Dictionary to hold data loaders for each subset
    globals.trainloaders = []
    globals.testloaders = []
    globals.valloaders = []
    subset_indices = []
    # Loop over each class pair
    for i, t in enumerate(class_pairs):
        # Get indices of images belonging to the specified class pair
        subs_ind = [idx for idx, (_, label) in enumerate(globals.trainset) if label in list(t)]
        val_subset_indices = [idx for idx, (_, label) in enumerate(valset) if label in list(t)]
        test_subset_indices = [idx for idx, (_, label) in enumerate(globals.testset) if label in list(t)]
        # Create a subset for the current class pair
        train_subset = Subset(globals.trainset, subs_ind)
        globals.trainloaders.append(DataLoader(train_subset, batch_size=globals.BATCH_SIZE, shuffle=True, pin_memory=True, num_workers = 0))

        subset_indices.append(subs_ind)
        
        val_subset = Subset(valset, val_subset_indices)
        globals.valloaders.append(DataLoader(val_subset, batch_size=500, shuffle=False))

        test_subset = Subset(globals.testset, test_subset_indices)
        globals.testloaders.append(DataLoader(test_subset, batch_size=500, shuffle=False))


In [5]:
def run_experiment(
        verbose = False,
        stopOnLoss = 0.02,
        full_CE = True,
        with_OOD = False,
        ood_method = 'jigsaw',
        kd_loss = 0,
        stopOnValAcc = None,
        epochs = 1000000,
        with_dropout = False,
        ogd = False
        ):
    def _print(*args, **kwargs):
        if verbose:
            print(*args, **kwargs)
    if with_OOD:
        globals.toggle_OOD(ood_method)
    else:
        globals.disable_OOD()
    initialize_data()
    prevModel = None
    globals.BATCH_SIZE=4
    
    globals.WITH_DROPOUT = with_dropout

    #[Denis] added code:
    Feature_Importance_Eval=Feature_Importance_Evaluations(globals.testloaders, DEVICE)

    for i in tqdm(range(globals.ITERATIONS), desc="Experiment Progress"):
        model = Net((i+1)*(globals.CLASSES_PER_ITER+globals.OOD_CLASS))
        if prevModel is not None:
            with torch.no_grad():
                model.copyPrev(prevModel)
        train_loader = globals.trainloaders[i]
        val_loader = globals.valloaders[i]
        if prevModel:
            _print("CL TRAIN!!")
            train_model_CL(
                model,
                prevModel,
                train_loader,
                val_loader,
                i,
                verbose,
                epochs,
                True,
                freeze_nonzero_params=False,
                l1_loss=0,
                ewc_loss=0,
                kd_loss=kd_loss,
                distance_loss=0,
                center_loss=0,
                param_reuse_loss=0,
                stopOnLoss=stopOnLoss,
                stopOnValAcc = stopOnValAcc,
                full_CE=full_CE,
                ogd=ogd,
                )
        else:
            train_model(
                model, 
                train_loader, 
                val_loader, 
                verbose, 
                epochs=epochs, 
                l1_loss=0,
                stopOnLoss=stopOnLoss,
                center_loss =0,
                ogd=ogd
                )

        #[Denis] added code:
        Feature_Importance_Eval.Task_Feature_Attribution(model, i)
        
        if verbose or i == globals.ITERATIONS-1:
            _print("Starting evaluation")
            _print("ITERATION", i+1)
            _print("ACCURACIES PER TASK:")
            accumPred = []
            all_labels = []
            all_embeddings = []
            with torch.no_grad():
                for j in range(i+1):
                    val_loader = globals.testloaders[j]
                    val_labels = get_labels(val_loader).to(DEVICE)
                    all_labels.append(val_labels)
                    model.eval()
                    pred, embeddings = model.get_pred_and_embeddings((get_features(val_loader).to(DEVICE)))
                    model.train()
                    accumPred.append(pred)
                    all_embeddings.append(embeddings)
                    sliced_pred = pred[:, j*(globals.CLASSES_PER_ITER+globals.OOD_CLASS):(j+1)*(globals.CLASSES_PER_ITER+globals.OOD_CLASS)]
                    _, predicted = torch.max(sliced_pred, 1)  # Get the class predictions
                    predicted += j*globals.CLASSES_PER_ITER
                    correct = (predicted == val_labels).sum().item()  # Count how many were correct
                    accuracy = correct / val_labels.size(0)  # Accuracy as a percentage
                    _print(str(accuracy), end=' ')
            accumPred = torch.cat(accumPred)
            all_labels = torch.cat(all_labels)
            all_embeddings = torch.cat(all_embeddings)
            predicted = []
            for x in accumPred:
                if globals.OOD_CLASS == 1:
                    x_pred = x[[i for i in range(x.size(0)) if (i + 1) % (globals.CLASSES_PER_ITER+1) != 0]]
                else:
                    x_pred = x
                x_pred = torch.softmax(x_pred, dim=-1)
                max = 0
                for (k, v) in enumerate(x_pred):
                    if v > max:
                        max = v
                        p = k
                predicted.append(p)
            predicted = torch.tensor(predicted).to(DEVICE)
            correct = (predicted == all_labels).sum().item()  # Count how many were correct
            accuracy = correct / all_labels.size(0)  # Accuracy as a percentage
            _print("Accuracy on tasks so far:", accuracy)

            embedding_drift = measure_embedding_drift(all_embeddings, all_labels, model.prev_test_embedding_centers)
            _print("Average embedding drift based on centroids:", embedding_drift)
            total_confusion, intra_phase_confusion, per_task_confusion = measure_embedding_confusion_knn(all_embeddings, all_labels, k = 1000, task=i+1)
            _print("Total confusion", total_confusion)
            _print("Intra-phase confusion", intra_phase_confusion)
            _print("Per task confusions", per_task_confusion)
            if verbose:
                plot_confusion_matrix(predicted.cpu(), all_labels.cpu(), list(range(globals.CLASSES_PER_ITER*(i+1))))
        prevModel = copy.deepcopy(model)
        
    #[Denis] added code:
    [avg_att_diff,att_diffs,_,_,avg_att_spread,att_spreads]=Feature_Importance_Eval.Get_Feature_Change_Score(prevModel)
    _print("Average SHAPC values (ordered as tasks):", att_diffs)
    _print("Averaged SHAPC value (the smaller the better):", avg_att_diff)
    _print("Average attention spread values (ordered as tasks):", att_spreads)
    _print("Averaged attention spread value (the bigger the better):", avg_att_spread)
    Feature_Importance_Eval.Save_Random_Picture_Salency() #prints the salcency maps for 1 example by class (first row: image, second row: salency map after training, third row: salency map after training task where class is included)
    
    return accuracy, total_confusion, intra_phase_confusion, per_task_confusion, embedding_drift, avg_att_diff, avg_att_spread

In [ ]:
def run_experiments(n_runs=globals.EXPERIMENT_N_RUNS, *args, **kwargs):
    verbose = kwargs.get('verbose', None)
    def _print(*args, **kwargs):
        if verbose:
            print(*args, **kwargs)
    def report_stats(data, name):
        #print(name, data)
        mean = statistics.mean(data)
        std = statistics.stdev(data)
        print(f"Mean " + name + f" across {n_runs} runs: {mean}")
        print(f"Standard deviation of " + name + f" across {n_runs} runs: {std}\n")
    accuracies = []
    total_confusions = []
    intra_phase_confusions = []
    per_task_confusions = []
    att_diffs = []
    embedding_drifts = []
    att_spreads = []

    jobs = []
    for r in range(n_runs):
        print(f"Starting run {r+1}.")
        # jobs.append(ex_parallel.submit(run_experiment,*args, **kwargs))
        jobs.append(run_experiment(*args, **kwargs))
    for i, job in enumerate(jobs):
        accuracy, total_confusion, intra_phase_confusion, per_task_confusion, embedding_drift, avg_att_diff, avg_att_spread = job.result()
        accuracies.append(accuracy)
        total_confusions.append(total_confusion)
        intra_phase_confusions.append(intra_phase_confusion)
        per_task_confusions.append(per_task_confusion)
        att_diffs.append(avg_att_diff)
        embedding_drifts.append(embedding_drift)
        att_spreads.append(avg_att_spread)
        print(f"Run {i} finished with accuracy {accuracy}")

    report_stats(accuracies, "accuracy")
    report_stats(total_confusions, "total confusion")
    report_stats(intra_phase_confusions, "intra-phase confusion")
    report_stats(per_task_confusions, "per-task confusion")
    report_stats(embedding_drifts, "embedding drift")
    report_stats(att_diffs, "attention drift")
    report_stats(att_spreads, "attention spread")

In [ ]:
run_experiments()

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Mean accuracy across 20 runs: 0.19729
Standard deviation of accuracy across 20 runs: 0.0003093286448525514

Mean total confusion across 20 runs: 0.414515965
Standard deviation of total confusion across 20 runs: 0.01604952756728125

Mean intra-phase confusion across 20 runs: 0.407553005
Standard deviation of intra-phase confusion across 20 runs: 0.015778470434289985

Mean per-task confusion across 20 runs: 0.08277860201892084
Standard deviation of per-task confusion across 20 runs: 0.007983167678031429

Mean embedding drift across 20 runs: 9.900845670700074
Standard deviation of embedding drift across 20 runs: 0.4113766512724969

Mean attention drift across 20 r

In [8]:
run_experiments(with_dropout=True)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Mean accuracy across 20 runs: 0.19776
Standard deviation of accuracy across 20 runs: 0.00012732056517228836

Mean total confusion across 20 runs: 0.447576495
Standard deviation of total confusion across 20 runs: 0.02142939851019906

Mean intra-phase confusion across 20 runs: 0.43697244
Standard deviation of intra-phase confusion across 20 runs: 0.020965855806761014

Mean per-task confusion across 20 runs: 0.11613773286171956
Standard deviation of per-task confusion across 20 runs: 0.011780864824994837

Mean embedding drift across 20 runs: 8.883338165283202
Standard deviation of embedding drift across 20 runs: 0.3353551149334345

Mean attention drift across 20 r

In [9]:
run_experiments(kd_loss=1)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Mean accuracy across 20 runs: 0.20146
Standard deviation of accuracy across 20 runs: 0.006270180052816414

Mean total confusion across 20 runs: 0.378874945
Standard deviation of total confusion across 20 runs: 0.010659788842308506

Mean intra-phase confusion across 20 runs: 0.37555696
Standard deviation of intra-phase confusion across 20 runs: 0.010214348913964638

Mean per-task confusion across 20 runs: 0.06480042035742914
Standard deviation of per-task confusion across 20 runs: 0.0066560068670996385

Mean embedding drift across 20 runs: 10.568913745880128
Standard deviation of embedding drift across 20 runs: 0.4949920468295522

Mean attention drift across 20 

In [10]:
run_experiments(full_CE=False)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Mean accuracy across 20 runs: 0.623225
Standard deviation of accuracy across 20 runs: 0.0612067580977959

Mean total confusion across 20 runs: 0.42478873500000003
Standard deviation of total confusion across 20 runs: 0.015463924293633774

Mean intra-phase confusion across 20 runs: 0.4224134
Standard deviation of intra-phase confusion across 20 runs: 0.01526683802622361

Mean per-task confusion across 20 runs: 0.06788457927885161
Standard deviation of per-task confusion across 20 runs: 0.005847927349584258

Mean embedding drift across 20 runs: 5.491150999069214
Standard deviation of embedding drift across 20 runs: 0.29895133171735105

Mean attention drift across

In [11]:
run_experiments(with_OOD=True)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Mean accuracy across 20 runs: 0.19753
Standard deviation of accuracy across 20 runs: 0.001541393764371096

Mean total confusion across 20 runs: 0.348665625
Standard deviation of total confusion across 20 runs: 0.0200980931129214

Mean intra-phase confusion across 20 runs: 0.341661555
Standard deviation of intra-phase confusion across 20 runs: 0.017989925005591774

Mean per-task confusion across 20 runs: 0.07377787103441767
Standard deviation of per-task confusion across 20 runs: 0.012119888500722348

Mean embedding drift across 20 runs: 11.41977047920227
Standard deviation of embedding drift across 20 runs: 0.6377873246918927

Mean attention drift across 20 run

In [12]:
run_experiments(ogd=True)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Mean accuracy across 20 runs: 0.198135
Standard deviation of accuracy across 20 runs: 0.003945853919989274

Mean total confusion across 20 runs: 0.40074635000000003
Standard deviation of total confusion across 20 runs: 0.014681977910546968

Mean intra-phase confusion across 20 runs: 0.39745372
Standard deviation of intra-phase confusion across 20 runs: 0.014508483057620572

Mean per-task confusion across 20 runs: 0.07140316540583184
Standard deviation of per-task confusion across 20 runs: 0.005296685010696834

Mean embedding drift across 20 runs: 10.762645626068116
Standard deviation of embedding drift across 20 runs: 0.8291818018837278

Mean attention drift ac

In [13]:
run_experiments(with_dropout=True, kd_loss=1, full_CE=False, with_OOD=True, ogd=True) # best version

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Mean accuracy across 20 runs: 0.86777
Standard deviation of accuracy across 20 runs: 0.013602867344791695

Mean total confusion across 20 runs: 0.310928865
Standard deviation of total confusion across 20 runs: 0.0067708420616232985

Mean intra-phase confusion across 20 runs: 0.30854194
Standard deviation of intra-phase confusion across 20 runs: 0.0066880282271974305

Mean per-task confusion across 20 runs: 0.050800132754598215
Standard deviation of per-task confusion across 20 runs: 0.0018811041010954434

Mean embedding drift across 20 runs: 4.782956767082214
Standard deviation of embedding drift across 20 runs: 0.2704109716467063

Mean attention drift across 2

In [22]:
run_experiments(with_dropout=True, kd_loss=1, full_CE=False, with_OOD=True, ogd=False)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Run 0 finished with accuracy 0.8527
Run 1 finished with accuracy 0.8573
Run 2 finished with accuracy 0.8746
Run 3 finished with accuracy 0.8791
Run 4 finished with accuracy 0.8469
Run 5 finished with accuracy 0.8244
Run 6 finished with accuracy 0.8274
Run 7 finished with accuracy 0.8593
Run 8 finished with accuracy 0.8573
Run 9 finished with accuracy 0.8778
Run 10 finished with accuracy 0.8343
Run 11 finished with accuracy 0.8767
Run 12 finished with accuracy 0.8693
Run 13 finished with accuracy 0.8545
Run 14 finished with accuracy 0.8693
Run 15 finished with accuracy 0.859
Run 16 finished with accuracy 0.851
Run 17 finished with accuracy 0.8699
Run 18 finished

In [23]:
run_experiments(with_dropout=False, kd_loss=1, full_CE=False, with_OOD=True, ogd=True)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Run 0 finished with accuracy 0.8399
Run 1 finished with accuracy 0.8056
Run 2 finished with accuracy 0.7323
Run 3 finished with accuracy 0.8221
Run 4 finished with accuracy 0.7856
Run 5 finished with accuracy 0.8099
Run 6 finished with accuracy 0.7452
Run 7 finished with accuracy 0.7821
Run 8 finished with accuracy 0.7926
Run 9 finished with accuracy 0.7991
Run 10 finished with accuracy 0.8117
Run 11 finished with accuracy 0.7414
Run 12 finished with accuracy 0.8436
Run 13 finished with accuracy 0.8162
Run 14 finished with accuracy 0.8319
Run 15 finished with accuracy 0.7562
Run 16 finished with accuracy 0.7717
Run 17 finished with accuracy 0.8371
Run 18 finish

In [24]:
run_experiments(with_dropout=True, kd_loss=1, full_CE=False, with_OOD=False, ogd=True)

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Run 0 finished with accuracy 0.8119
Run 1 finished with accuracy 0.7824
Run 2 finished with accuracy 0.7514
Run 3 finished with accuracy 0.7462
Run 4 finished with accuracy 0.7386
Run 5 finished with accuracy 0.7901
Run 6 finished with accuracy 0.76
Run 7 finished with accuracy 0.7492
Run 8 finished with accuracy 0.8001
Run 9 finished with accuracy 0.7887
Run 10 finished with accuracy 0.696
Run 11 finished with accuracy 0.7436
Run 12 finished with accuracy 0.7926
Run 13 finished with accuracy 0.7333
Run 14 finished with accuracy 0.7811
Run 15 finished with accuracy 0.7462
Run 16 finished with accuracy 0.7443
Run 17 finished with accuracy 0.7516
Run 18 finished 

In [25]:
# training on entire dataset
globals.ITERATIONS = 1
globals.CLASSES_PER_ITER = 10
run_experiments()

Starting run 1.
Starting run 2.
Starting run 3.
Starting run 4.
Starting run 5.
Starting run 6.
Starting run 7.
Starting run 8.
Starting run 9.
Starting run 10.
Starting run 11.
Starting run 12.
Starting run 13.
Starting run 14.
Starting run 15.
Starting run 16.
Starting run 17.
Starting run 18.
Starting run 19.
Starting run 20.
Run 0 finished with accuracy 0.1967
Run 1 finished with accuracy 0.197
Run 2 finished with accuracy 0.1976
Run 3 finished with accuracy 0.1957
Run 4 finished with accuracy 0.1968
Run 5 finished with accuracy 0.1975
Run 6 finished with accuracy 0.1977
Run 7 finished with accuracy 0.1939
Run 8 finished with accuracy 0.1976
Run 9 finished with accuracy 0.1974
Run 10 finished with accuracy 0.1965
Run 11 finished with accuracy 0.1976
Run 12 finished with accuracy 0.1974
Run 13 finished with accuracy 0.1974
Run 14 finished with accuracy 0.1975
Run 15 finished with accuracy 0.1973
Run 16 finished with accuracy 0.1969
Run 17 finished with accuracy 0.1976
Run 18 finishe

In [17]:
torch.cuda.empty_cache()